In [1]:
# from transformers import LlamaForCausalLM, LlamaTokenizer

# tokenizer = LlamaTokenizer.from_pretrained("/mnt/disk-1/llama_hf_7B")
# model = LlamaForCausalLM.from_pretrained("/mnt/disk-1/llama_hf_7B")

import transformers
from transformers import pipeline
from accelerate import Accelerator

accelerator = Accelerator()

model = transformers.AutoModelForCausalLM.from_pretrained("/mnt/disk-1/alpaca-7b")
tokenizer = transformers.AutoTokenizer.from_pretrained("/mnt/disk-1/alpaca-7b")
model, tokenizer = accelerator.prepare(model, tokenizer)

/home/yuxizheng/miniconda3/envs/get_responses/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.21s/it]


In [2]:
from transformers import pipeline

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=accelerator.device)

# batches = ["Answer with only one number from 1 to 7, considering 1 as 'extremely dissimilar', 2 as 'very dissimilar', 3 as 'likely dissimilar', 4 as 'neutral', 5 as 'likely similar', 6 as 'very similar', and 7 as 'extremely similar': How similar is Alligator and Alligator?",
#              "Answer with only one number from 1 to 7, considering 1 as 'extremely dissimilar', 2 as 'very dissimilar', 3 as 'likely dissimilar', 4 as 'neutral', 5 as 'likely similar', 6 as 'very similar', and 7 as 'extremely similar': How similar is Alligator and Boapython?",
#              "Answer with only one number from 1 to 7, considering 1 as 'extremely dissimilar', 2 as 'very dissimilar', 3 as 'likely dissimilar', 4 as 'neutral', 5 as 'likely similar', 6 as 'very similar', and 7 as 'extremely similar': How similar is Alligator and Blindworm?",
#              "Answer with only one number from 1 to 7, considering 1 as 'extremely dissimilar', 2 as 'very dissimilar', 3 as 'likely dissimilar', 4 as 'neutral', 5 as 'likely similar', 6 as 'very similar', and 7 as 'extremely similar': How similar is Alligator and Caiman?"]

# response = pipe(question[0], max_length=100, do_sample=False, temperature=0.1)
batches = ["Answer using only one word - Alligator or Spanner and not Shovel. Which is more similar in meaning to Shovel?",
"Answer using only one word - Caiman or Tortoise and not Anvil. Which is more similar in meaning to Anvil?",
"Answer using only one word - Boa python or Snake and not Nail. Which is more similar in meaning to Nail?",
"Answer using only one word - Chisel or Toad and not Paint brush. Which is more similar in meaning to Paint brush?"
"Answer using only one word - Caiman or Crocodile and not Shovel. Which is more similar in meaning to Shovel?"]


# for response in responses:
#     print(response)
responses = []
prompt_and_response = [] 

# for prompt in batches:
#     output = pipe(batches, max_length=50, do_sample=False)[0] # Set your own max_length and other parameters
#     response = output['generated_text']
#     responses.append(response)
responses = pipe(batches, max_new_tokens=20, do_sample=False)
responses = [r[0]['generated_text'] for r in responses]
responses = [','.join(r.split('\n')[1::]) for r in responses]

for prompt, response in zip(batches, responses):
    prompt_and_response.append([prompt, response])

prompt_and_response

[['Answer using only one word - Alligator or Spanner and not Shovel. Which is more similar in meaning to Shovel?',
  'A) Alligator,B) Spanner,C) Hammer,D) S'],
 ['Answer using only one word - Caiman or Tortoise and not Anvil. Which is more similar in meaning to Anvil?',
  'Caiman or Tortoise.'],
 ['Answer using only one word - Boa python or Snake and not Nail. Which is more similar in meaning to Nail?',
  'Boa python or Snake and not Nail. Which is more similar in meaning to N'],
 ['Answer using only one word - Chisel or Toad and not Paint brush. Which is more similar in meaning to Paint brush?Answer using only one word - Caiman or Crocodile and not Shovel. Which is more similar in meaning to Shovel?',
  '']]

In [2]:



def get_llama_responses(batches, model_type):
    responses = []
    prompt_and_response = [] 
    batches = list(itertools.chain(*batches))

    # get the responses
    for prompt in batches:
        output = pipe(prompt, max_length=50, do_sample=False)[0] # Set your own max_length and other parameters
        response = output['generated_text']
        responses.append(response)

    # return the results
    for prompt, response in zip(batches, responses):
        prompt_and_response.append([prompt, response])
        
    return prompt_and_response

get_llama_responses(questions, "alpaca")

KeyboardInterrupt: 